In [1]:
import sys, os, json, subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
import sit_helper
standard_import_tool_plugin_path=sit_helper.load_standard_import_tool_plugin()
import cbm3_python_helper
cbm3_python_helper.load_cbm3_python()

import cbm3_python.simulation.projectsimulator
from cbm3_python.cbm3data import cbm3_results
from cbm3_python.cbm3data.aidb import AIDB
from cbm3_python.cbm3data.accessdb import AccessDB
from cbm3_python.cbm3data.projectdb import ProjectDB
from cbm3_python.simulation.simulator import Simulator
from cbm3_python.cbm3data import cbm3_results

In [3]:
local_working_dir = os.path.abspath(".")
toolbox_path = r'C:\Program Files (x86)\Operational-Scale CBM-CFS3'
aidb_path=os.path.join(toolbox_path, "admin", "dbs", "ArchiveIndex_Beta_Install.mdb")
cbm_exe_path=os.path.join(toolbox_path, "admin", "executables")
cbm_wd = os.path.join(toolbox_path, "temp")

#the number of steps in each iteration
iteration_steps = 10
num_iterations = 5

In [4]:
def create_event_factory(events):
    def factory(config):
        for e in events:
            config.add_event(**e)
    return factory
            
def create_transition_rule_factory(transitions):
    def factory(config):
        for t in transitions:
            config.add_transition_rule(**t)
    return factory

In [5]:
def iteration_import(project_path, config_save_path, events_factory, transition_factory):


    c = sit_helper.SITConfig(
        imported_project_path=project_path,
        initialize_mapping=True
    )
    c.data_config(
        age_class_size=5,
        num_age_classes=14,
        classifiers=["Stand ID", "Leading Species", "Land Cover"])

    c.set_single_spatial_unit(17)
    c.set_species_classifier("Leading Species")
    c.map_species("Nonforest", "Not stocked")

    c.add_inventory(classifier_set=["Stand 1", "Luvisolic (E. Canada)", "Nonforest"],
                    area=100, age=0, unfccc_land_class=1)
    c.add_inventory(classifier_set=["Stand 2", "Spruce", "Forest Only"],
                    area=1000, age=0, unfccc_land_class=0,
                   historic_disturbance="Wildfire", last_pass_disturbance="Wildfire")

    growth_curve_data = [0,0,0,27,84,155,217,271,317,353,382,406,426,426]

    not_stocked_growth = [0.0]*len(growth_curve_data)

    c.add_yield(classifier_set=["?","?","Nonforest"],
                leading_species_classifier_value="Not stocked",
                values=not_stocked_growth)

    c.add_yield(classifier_set=["?","Red pine","Forest Only"], 
                leading_species_classifier_value="Red pine", values=growth_curve_data)
    
    c.add_yield(classifier_set=["?","Spruce","Forest Only"], 
                leading_species_classifier_value="Spruce", values=growth_curve_data)
    
    events_factory(c)
    transition_factory(c)
    
    c.import_project(standard_import_tool_plugin_path, config_save_path)

    

In [6]:
def get_project_path(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_{}.mdb".format(iteration_num))

def get_results_db_path(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_results_{}.mdb".format(iteration_num))

def get_config_save_path(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_config_{}".format(iteration_num))

def get_iteration_tempfiles_dir(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_tempfiles_{}".format(iteration_num))


In [7]:
events = [
    {
    "classifier_set": ["?","?","?"],
    "disturbance_type": "Afforestation", 
    "time_step": 1, 
    "target": 100,
    "target_type": "Area",
    "sort": "SORT_BY_SW_AGE"
    }
]
transitions = [
    {
    "classifier_set_source": ["?","?","?"],
    "classifier_set_target": ["?","Red pine","Forest Only"],
    "disturbance_type": "Afforestation",
    "percent": 100
    }
]

iteration_num = 1
project_path = get_project_path(iteration_num)
tempfiles_dir = get_iteration_tempfiles_dir(iteration_num)
results_db_path = get_results_db_path(iteration_num)

iteration_import(
    project_path,
    get_config_save_path(iteration_num),
    create_event_factory(events),
    create_transition_rule_factory(transitions))



In [8]:
with AIDB(aidb_path, False) as aidb, \
     AccessDB(project_path, False) as proj:

    proj.ExecuteQuery("UPDATE tblRunTableDetails SET tblRunTableDetails.RunLength = ?", iteration_steps)
    simId = aidb.AddProjectToAIDB(proj)
    s = Simulator(cbm_exe_path,
                simId,
                os.path.dirname(project_path),
                cbm_wd,
                toolbox_path)
    

    s.CleanupRunDirectory()
    s.CreateMakelistFiles()
    s.copyMakelist()
    s.runMakelist()
    s.loadMakelistSVLS()
    s.DumpMakelistSVLs()
    s.CreateCBMFiles()
    s.CopyCBMExecutable()
    s.RunCBM()
    s.CopyTempFiles(output_dir=tempfiles_dir)
    s.LoadCBMResults(output_path = results_db_path)

now run subsequent iterations

In [9]:

for i in range(2,num_iterations+1):
    iteration_num = i
    project_path = get_project_path(iteration_num)
    tempfiles_dir = get_iteration_tempfiles_dir(iteration_num)
    results_db_path = get_results_db_path(iteration_num)

    #get the total biomass from a previous iteration
    total_biomass = cbm3_results.load_pool_indicators( get_results_db_path(i-1))["Total Biomass"][-1:].values[0]
    #set the target to 5% of the total biomass from the previous iteration
    targetC = total_biomass * 0.05
    
    events = [
        {
        "classifier_set": ["?","?","?"],
        "disturbance_type": "Clear-cut with slash-burn", 
        "time_step": x+1, 
        "target": targetC/iteration_steps,
        "target_type": "Merchantable",
        "sort": "MERCHCSORT_TOTAL"
        } for x in range(iteration_steps)
    ]
    
    iteration_import(
        project_path,
        get_config_save_path(iteration_num),
        create_event_factory(events),
        create_transition_rule_factory([]))
    
    with AIDB(aidb_path, False) as aidb, \
         AccessDB(project_path, False) as proj:

        proj.ExecuteQuery("UPDATE tblRunTableDetails SET tblRunTableDetails.RunLength = ?", iteration_steps)
        simId = aidb.AddProjectToAIDB(proj)
        s = Simulator(cbm_exe_path,
                    simId,
                    os.path.dirname(project_path),
                    cbm_wd,
                    toolbox_path)

        s.CleanupRunDirectory()
        s.CopyToWorkingDir(project_path)
        s.CreateCBMFiles()
        s.CopyCBMExecutable()
        s.CopySVLFromPreviousRun(os.path.join(get_iteration_tempfiles_dir(i-1),"CBMRun","output"))
        s.RunCBM()
        s.CopyTempFiles(output_dir=tempfiles_dir)
        s.LoadCBMResults(output_path = results_db_path)
   

In [145]:
cbm3_results.load_pool_indicators( get_results_db_path(1))["Total Biomass"]
    

0         0.000000
1       102.647662
2       448.064649
3      1059.966821
4      1950.202115
5      3125.129825
6      4587.288546
7      6336.124666
8      8368.397713
9     10678.445329
10    13258.388015
Name: Total Biomass, dtype: float64

In [146]:
cbm3_results.load_pool_indicators( get_results_db_path(2))["Total Biomass"]
    

0     13258.388015
1     16098.308465
2     19186.416672
3     22509.213921
4     26051.666490
5     29797.375394
6     33728.761085
7     37827.240095
8     42073.429556
9     46447.325253
10    50928.500982
Name: Total Biomass, dtype: float64

In [147]:
cbm3_results.load_pool_indicators( get_results_db_path(3))["Total Biomass"]

0     50928.500982
1     55496.306383
2     60808.829333
3     66149.865759
4     71427.624977
5     76650.678663
6     81172.518959
7     85662.114870
8     90122.990392
9     94558.255256
10    98970.628297
Name: Total Biomass, dtype: float64

In [149]:
cbm3_results.load_pool_indicators( get_results_db_path(4))["Total Biomass"]

0      98970.628297
1     102796.928621
2     106609.200966
3     110408.779873
4     114196.904791
5     117974.741168
6     121185.609936
7     124390.372782
8     127589.668638
9     130784.030323
10    133973.982354
Name: Total Biomass, dtype: float64

In [150]:
cbm3_results.load_pool_indicators( get_results_db_path(5))["Total Biomass"]

0     133973.982354
1     136467.732550
2     138959.326201
3     141449.022531
4     143905.856649
5     146329.367287
6     148276.914940
7     150313.283608
8     152364.988367
9     154412.267933
10    156455.238629
Name: Total Biomass, dtype: float64